In [76]:
import numpy as np
import tensorflow as tf
import nltk
import itertools
from gensim.models import Word2Vec

In [10]:
np.random.multinomial(1, [0.125,0.25,0.25, 0.125, 0.125], size=1)

array([[0, 0, 1, 0, 0]])

In [1]:
# Batch processifile_data = open('gh_create_all_fields_cleaned_data.txt', 'rt')
# reading the sentences:
file_data = open('gh_create_all_fields_cleaned_data.txt', 'rt')
# reading the sentences:
sentences=file_data.read()
sentences=sentences.split('\n')




In [3]:
out, inp = [" ".join(x.split()[1:]) for x in sentences], [" ".join(x.split()[:1]) for x in sentences]

In [6]:
inp_table=[]
for word in inp:
    inp_table.append('$'+word+'&')

In [13]:
#padding:
maxLen= len(max(inp_table, key=len))

for index in range(len(inp_table)):
    inp_table[index]= inp_table[index]+'0'*(maxLen-len(inp_table[index]))
    
    

In [82]:
inp_table

['$abalone&000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '$abalone&000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '$abbreviationoffsets&000000000000000000000000000000000000000000000000000000000000000000',
 '$abc&0000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '$abc&0000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '$abcd&000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '$abc&0000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '$abookgroup&000000000000000000000000000000000000000000000000000000000000000000000000000',
 '$about&00000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '$about&00000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '$about&00000000000000000000000000000000000000000000000000000000000000000000000

In [15]:
char_data =""

sent=""
for index in range(len(inp_table)):
    sent+= inp_table[index]+" "+out[index]+"\n"


# for sent in train_sent:
#     char_data+=sent+"\n"

chars= list(set(sent))

#create dictionary mapping for each char
char_to_ix = {ch:i for (i,ch) in enumerate(chars)}
ix_to_char = {i:ch for (i,ch) in enumerate(chars)}

In [18]:
char_data=[]

for word in inp_table:
    temp=[]
    for char in word:
        temp.append(char_to_ix[char])
    char_data.append(temp)

In [81]:
# char_data

In [68]:

for index in range(0, len(char_data), 50):
    batch = char_data[index:index+50]
    batch = np.array([np.array(xi) for xi in batch])
    batch = tf.convert_to_tensor(batch, dtype=tf.float32)
#     op = tf.gather(batch, [0,6], axis=1)
    
    print(batch[:,0:len(batch[0])-1])
    break


tf.Tensor(
[[11.  3. 12. ...  2.  2.  2.]
 [11.  3. 12. ...  2.  2.  2.]
 [11.  3. 12. ...  2.  2.  2.]
 ...
 [11.  3. 23. ...  2.  2.  2.]
 [11.  3. 23. ...  2.  2.  2.]
 [11.  3. 23. ...  2.  2.  2.]], shape=(50, 86), dtype=float32)


In [47]:
from d2l import tensorflow as d2l


batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [53]:
for x,y in train_iter:
    print(x, y)
    break

tf.Tensor(
[[ 5  3  9 ... 12 12  1]
 [ 9  2 11 ...  1 15  9]
 [ 2  1  8 ...  2  8  3]
 ...
 [ 3  1  4 ... 12  2 13]
 [ 2  6  3 ...  1  5  1]
 [ 1  3  9 ... 15  9  4]], shape=(32, 35), dtype=int32) tf.Tensor(
[[ 3  9  2 ... 12  1 21]
 [ 2 11  1 ... 15  9  4]
 [ 1  8  4 ...  8  3  6]
 ...
 [ 1  4  1 ...  2 13  1]
 [ 6  3 15 ...  5  1 13]
 [ 3  9  2 ...  9  4  5]], shape=(32, 35), dtype=int32)


# making the bag of characters model

In [70]:
# functions to calculate word tokenizations and tokenizers:
# which would be later used for calculating the word to 

def filter_words(sentence):
    ll = [[nltk.word_tokenize(w), ' '] for w in sentence.split()]
#     print(list(itertools.chain(*list(itertools.chain(*ll)))))
#     return [word.lower() if word.isalpha() else " " for word in [nltk.word_tokenize(sentence), " "]]
    return list(itertools.chain(*list(itertools.chain(*ll))))

def tokenize(text):
    return [filter_words(sentence) for sentence in nltk.sent_tokenize(text)]


In [77]:
tokenized_words=[]
sent_data=[]
for sentence in out:
    sent_mod=''
    for char in sentence:
        sent_mod+=char+' ' 
#     print(sent_mod)
#     print(nltk.sent_tokenize(sent_mod))
    tokenized_words.extend(tokenize(sent_mod))
    sent_data.append(sentence)

word2vec_model = Word2Vec(tokenized_words, 
                          sg=1,  # here we will use skipgram 0 is for CBROW; skip gram is for smaller models 
                          size=10,
                          min_count=1 # word needs to occur atleast once
                         )

In [79]:
def my_func(arg):
    arg = tf.convert_to_tensor(arg, dtype=tf.float32)
    return arg

column2vec=[]

for column in out:
    result=tf.zeros([len(chars), 1], tf.float32)
    for every_char in column:
        tfcharEmb = my_func(word2vec_model[every_char])
        tfcharEmb=tf.reshape(tfcharEmb, [10, 1])
        result+= tf.matmul(E, tfcharEmb)
        
    column2vec.append(result)

/Users/kedargn94/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


NameError: name 'E' is not defined